In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- 1. VERİ YÜKLEME VE ETİKETLEME (FEATURE ENGINEERING) ---
def load_and_label_data():
    # Dosya İsimleri ve Tanımladığımız "BIOS Ayarları" (Manuel Etiketleme)
    # Bu ayarlar modelin "Input"u olacak.
    files_config = {
        "Fans ve apu down.csv": {
            "Label": "Default",
            "Set_PPT": 142,  # Ryzen 7600X Stock PPT
            "Set_CO": 0,     # Curve Optimizer
            "Set_Boost": 0   # Boost Override
        },
        "-30 CCD PBO 95 +200 mhz.csv": {
            "Label": "PBO_Tuned_Max",
            "Set_PPT": 95,
            "Set_CO": -30,
            "Set_Boost": 200
        },
        "65W-85W_75A_150A-25_Per_CCD.csv": {
            "Label": "Eco_Mode_65W",
            "Set_PPT": 65,
            "Set_CO": -25,
            "Set_Boost": 0
        },
        "95W-115W_80A_140A-20_Per_CCD.csv": {
            "Label": "Performance_95W",
            "Set_PPT": 95,
            "Set_CO": -20,
            "Set_Boost": 0
        }
    }

    df_list = []
    
    print("Veriler Yükleniyor ve Etiketleniyor...")
    
    for fname, config in files_config.items():
        if os.path.exists(fname):
            # HWInfo dosyasını temizleyerek oku
            try:
                temp_df = pd.read_csv(fname, low_memory=False, on_bad_lines='skip')
                temp_df.columns = [c.strip() for c in temp_df.columns] # Boşluk temizliği
                
                # Sadece ML için gerekli sütunları alalım (Gürültüyü azaltmak için)
                # Girdiler (Inputs) ve Çıktılar (Targets)
                keep_cols = [
                    'CPU (Tctl/Tdie) [°C]',                # Hedef 1: Sıcaklık
                    'Çekirdek Etkili Saatler (avg) [MHz]', # Hedef 2: Performans
                    'CPU Tüm Güç Tüketimi [W]',            # Hedef 3: Güç
                    'CPU [RPM]',                           # Girdi: Fan Hızı
                    'CPU SoC Güç Tüketimi (SVI3 TFN) [W]'  # Girdi: SoC Yükü
                ]
                
                # Varsa al, yoksa geç
                available_cols = [c for c in keep_cols if c in temp_df.columns]
                temp_df = temp_df[available_cols].dropna() # Boş verileri at
                
                # Manuel Etiketleri (Features) Ekle
                temp_df['Set_PPT_Limit'] = config['Set_PPT']
                temp_df['Set_CO_Value'] = config['Set_CO']
                temp_df['Set_Boost_Override'] = config['Set_Boost']
                temp_df['Config_Label'] = config['Label'] # Görselleştirme için
                
                df_list.append(temp_df)
                print(f"  -> {fname} yüklendi ({len(temp_df)} satır). Ayarlar: PPT={config['Set_PPT']}W, CO={config['Set_CO']}")
                
            except Exception as e:
                print(f"HATA: {fname} okunamadı. Sebebi: {e}")
        else:
            print(f"UYARI: {fname} bulunamadı.")

    # Tüm veriyi birleştir
    full_df = pd.concat(df_list, ignore_index=True)
    return full_df

# --- 2. DATASET BÖLME (%75 Train - %25 Test) ---
df = load_and_label_data()

# Model Girdileri (X): Ayarlar + Çevresel Faktörler (Fan Hızı vb.)
# Fan hızını da "Girdi" olarak alıyoruz, böylece "Fan %50 olursa sıcaklık ne olur?" diye sorabileceğiz.
X = df[['Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', 'CPU [RPM]']]

# Model Hedefleri (Y): Tahmin etmek istediklerimiz
# 1. Sıcaklık Modeli için
y_temp = df['CPU (Tctl/Tdie) [°C]']
# 2. Performans Modeli için
y_clock = df['Çekirdek Etkili Saatler (avg) [MHz]']

# Bölme İşlemi (Random State 42 = Bilimsel Tekrarlanabilirlik için sabit tohum)
# Shuffle=True (Veriyi karıştır) önemlidir, çünkü zaman serisi verisinde
# "ilk dosyayı train, son dosyayı test" yaparsak model yanılır. Karıştırarak genel karakteristiği öğretiyoruz.
X_train, X_test, y_temp_train, y_temp_test, y_clock_train, y_clock_test = train_test_split(
    X, y_temp, y_clock, test_size=0.25, random_state=42
)

print("\n--- Veri Seti Hazır ---")
print(f"Toplam Veri Sayısı: {len(df)}")
print(f"Eğitim Seti (%75): {len(X_train)} örnek")
print(f"Test Seti (%25):   {len(X_test)} örnek")
print("\nMakine Öğrenimi Modelleri kurulmaya hazır.")

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# --- 3. MODEL EĞİTİMİ (TRAINING) ---
print("Modeller Eğitiliyor... (Bu işlem birkaç saniye sürebilir)")

# A) Termal Model (Sıcaklık Tahmincisi)
model_temp = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_temp.fit(X_train, y_temp_train)
print("✅ Termal Model Eğitildi.")

# B) Performans Modeli (MHz Tahmincisi)
model_clock = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
model_clock.fit(X_train, y_clock_train)
print("✅ Performans Modeli Eğitildi.")

# --- 4. MODEL TESTİ VE DEĞERLENDİRME (EVALUATION) ---
# Test seti üzerinde tahmin yap
pred_temp = model_temp.predict(X_test)
pred_clock = model_clock.predict(X_test)

# Metrikleri Hesapla
# MAE (Mean Absolute Error): Ortalama kaç derece/MHz sapıyoruz?
mae_temp = mean_absolute_error(y_temp_test, pred_temp)
mae_clock = mean_absolute_error(y_clock_test, pred_clock)

# R2 Score: Modelin veriyi açıklama başarısı (1.00 mükemmeldir)
r2_temp = r2_score(y_temp_test, pred_temp)
r2_clock = r2_score(y_clock_test, pred_clock)

print("\n--- MODEL PERFORMANS RAPORU ---")
print(f"🌡️  Termal Model Hatası (MAE): ±{mae_temp:.2f}°C")
print(f"🎯 Termal Model Başarısı (R²): {r2_temp:.4f}")
print("-" * 30)
print(f"🚀 Performans Model Hatası (MAE): ±{mae_clock:.2f} MHz")
print(f"🎯 Performans Model Başarısı (R²): {r2_clock:.4f}")

# --- 5. GÖRSELLEŞTİRME (Doğruluk ve Özellik Önemi) ---
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
sns.set_style("darkgrid")

# Grafik 1: Gerçek vs Tahmin (Sıcaklık)
sns.scatterplot(x=y_temp_test, y=pred_temp, ax=axes[0,0], alpha=0.5, color='orange')
axes[0,0].plot([y_temp.min(), y_temp.max()], [y_temp.min(), y_temp.max()], 'r--', lw=2) # İdeal çizgi
axes[0,0].set_title(f'Termal Tahmin Doğruluğu (R²={r2_temp:.3f})')
axes[0,0].set_xlabel('Gerçek Sıcaklık (°C)')
axes[0,0].set_ylabel('Tahmin Edilen Sıcaklık (°C)')

# Grafik 2: Gerçek vs Tahmin (Performans)
sns.scatterplot(x=y_clock_test, y=pred_clock, ax=axes[0,1], alpha=0.5, color='green')
axes[0,1].plot([y_clock.min(), y_clock.max()], [y_clock.min(), y_clock.max()], 'r--', lw=2)
axes[0,1].set_title(f'Performans Tahmin Doğruluğu (R²={r2_clock:.3f})')
axes[0,1].set_xlabel('Gerçek Frekans (MHz)')
axes[0,1].set_ylabel('Tahmin Edilen Frekans (MHz)')

# Grafik 3: Özellik Önemi (Sıcaklığı en çok ne etkiliyor?)
feat_imp_temp = pd.Series(model_temp.feature_importances_, index=X.columns).sort_values(ascending=False)
sns.barplot(x=feat_imp_temp.values, y=feat_imp_temp.index, ax=axes[1,0], palette='Oranges_r')
axes[1,0].set_title('Sıcaklığı Etkileyen Faktörler')

# Grafik 4: Özellik Önemi (Hızı en çok ne etkiliyor?)
feat_imp_clock = pd.Series(model_clock.feature_importances_, index=X.columns).sort_values(ascending=False)
sns.barplot(x=feat_imp_clock.values, y=feat_imp_clock.index, ax=axes[1,1], palette='Greens_r')
axes[1,1].set_title('Performansı Etkileyen Faktörler')

plt.tight_layout()
plt.show()

In [ ]:
def run_virtual_stress_test(model_t, model_c, current_worst_core_rank):
    """
    Eğitilmiş ML modellerini kullanarak, denenmemiş BIOS ayarlarını simüle eder.
    Ayrıca Silikon Kalitesi verisine göre çökme riski tahmini yapar.
    """
    
    # --- 1. SENARYOLAR (What-If Analysis) ---
    # Buraya denemek istediğiniz hayali ayarları giriyoruz.
    # Fan Hızı (RPM) olarak 1500 (Max) baz alınmıştır.
    scenarios = [
        # Senaryo 1: Aşırı Tasarruf (Laptop Modu gibi)
        {"Name": "Ultra Eco (45W)", "Set_PPT_Limit": 45, "Set_CO_Value": -30, "Set_Boost_Override": 0, "CPU [RPM]": 1000},
        
        # Senaryo 2: "Sweet Spot" Adayı (Analizlerden tahmin ettiğimiz ideal nokta)
        {"Name": "AI Optimized (85W)", "Set_PPT_Limit": 85, "Set_CO_Value": -25, "Set_Boost_Override": 100, "CPU [RPM]": 1500},
        
        # Senaryo 3: Riskli Performans (Limitleri zorla)
        {"Name": "Extreme OC (130W)", "Set_PPT_Limit": 130, "Set_CO_Value": -30, "Set_Boost_Override": 200, "CPU [RPM]": 2000},
        
        # Senaryo 4: İmkansız Undervolt (Çökme testi)
        {"Name": "Suicide Run (-40 CO)", "Set_PPT_Limit": 105, "Set_CO_Value": -40, "Set_Boost_Override": 150, "CPU [RPM]": 2000},
        
        # Senaryo 5: Sessiz Performans
        {"Name": "Silent Gaming", "Set_PPT_Limit": 95, "Set_CO_Value": -20, "Set_Boost_Override": 50, "CPU [RPM]": 800}
    ]
    
    # DataFrame oluştur
    sim_df = pd.DataFrame(scenarios)
    
    # --- 2. YAPAY ZEKA TAHMİNİ (PREDICTION) ---
    # Modelin beklediği özellik sırası: ['Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', 'CPU [RPM]']
    X_sim = sim_df[['Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', 'CPU [RPM]']]
    
    print("Yapay Zeka Senaryoları İşliyor...")
    
    # Tahminleri yap
    pred_temps = model_t.predict(X_sim)
    pred_clocks = model_c.predict(X_sim)
    
    # Sonuçları tabloya ekle
    sim_df['Tahmini Sıcaklık (°C)'] = pred_temps
    sim_df['Tahmini Hız (MHz)'] = pred_clocks
    sim_df['Verimlilik (MHz/°C)'] = sim_df['Tahmini Hız (MHz)'] / sim_df['Tahmini Sıcaklık (°C)']
    
    # --- 3. RİSK ANALİZİ (Logic Based on Silicon Lottery) ---
    # Core 5'in kalitesi düşük olduğu için agresif CO değerlerinde uyarı verelim.
    def assess_risk(row):
        risk_score = 0
        reasons = []
        
        # Risk 1: Curve Optimizer Sınırı
        # Bizim analizimizde Core 5 (-20) istiyordu. -25 ve altı risklidir.
        if row['Set_CO_Value'] < -25: 
            risk_score += 5
            reasons.append(f"ÇÖKME RİSKİ: Zayıf Çekirdek (Core 5) -25 altını kaldıramayabilir.")
        elif row['Set_CO_Value'] < -20:
            risk_score += 2
            reasons.append("Uyarı: Core 5 kararsızlaşabilir.")
            
        # Risk 2: Termal Sınır
        if row['Tahmini Sıcaklık (°C)'] > 92:
            risk_score += 3
            reasons.append("Termal Limit: 95°C'ye çok yakın.")
            
        # Risk 3: PPT Yetersizliği
        # Eğer PPT çok düşükse (45W) ve Boost yüksekse, saat hızı düşer (Clock Stretching).
        if row['Set_PPT_Limit'] < 65 and row['Set_Boost_Override'] > 0:
            reasons.append("Verimsiz: Düşük Watt ile Boost yapılamaz.")

        if risk_score >= 5: return "🔴 YÜKSEK RİSK (" + ", ".join(reasons) + ")"
        elif risk_score >= 2: return "🟡 ORTA RİSK (" + ", ".join(reasons) + ")"
        else: return "🟢 GÜVENLİ"

    sim_df['Risk Analizi'] = sim_df.apply(assess_risk, axis=1)
    
    # --- 4. SONUÇ GÖSTERİMİ ---
    display_cols = ['Name', 'Set_PPT_Limit', 'Set_CO_Value', 'Tahmini Sıcaklık (°C)', 'Tahmini Hız (MHz)', 'Risk Analizi']
    
    print("\n--- AI DESTEKLİ SANAL STRES TESTİ SONUÇLARI ---")
    display(sim_df[display_cols].style.background_gradient(cmap='RdYlGn_r', subset=['Tahmini Sıcaklık (°C)'])
                                     .background_gradient(cmap='RdYlGn', subset=['Tahmini Hız (MHz)'])
                                     .format({"Tahmini Sıcaklık (°C)": "{:.1f}", "Tahmini Hız (MHz)": "{:.0f}"}))

# Modelleri önceki hücreden (global) alarak çalıştır
# Worst Core Rank: Bizim analizde Core 5 en kötü çıkmıştı.
run_virtual_stress_test(model_temp, model_clock, current_worst_core_rank=5)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. SÜTUN İSİMLERİNİ TEMİZLEME (XGBoost Hatası Düzeltmesi) ---
# Köşeli parantezleri kaldırıyoruz: "CPU [RPM]" -> "CPU RPM" oluyor.
print("Sütun isimleri XGBoost için temizleniyor...")
df.columns = df.columns.str.replace('[', '', regex=False).str.replace(']', '', regex=False).str.strip()

# İsimler değiştiği için değişkenleri güncelleyelim
# HWInfo Türkçe karakterleri ve birimleri:
col_temp = 'CPU (Tctl/Tdie) °C' 
col_clock = 'Çekirdek Etkili Saatler (avg) MHz'
col_power = 'CPU Tüm Güç Tüketimi W'
col_rpm = 'CPU RPM' # Artık parantez yok

# --- 2. FEATURE ENGINEERING 2.0 (Zaman ve Fizik) ---
def create_advanced_features(input_df):
    df_adv = input_df.copy()
    
    print("Gelişmiş özellikler (Lag & Rolling) oluşturuluyor...")
    
    # Her profili kendi içinde grupla (Veri karışmasını önle)
    grouped = df_adv.groupby('Config_Label')
    
    # A) LAG FEATURES (Geçmiş Bilgisi)
    # 1 satır önceki sıcaklık ve hız
    df_adv['Temp_Lag1'] = grouped[col_temp].shift(1)
    df_adv['Temp_Lag2'] = grouped[col_temp].shift(2)
    df_adv['Clock_Lag1'] = grouped[col_clock].shift(1)
    
    # B) ROLLING FEATURES (Ortalama Trend)
    # Son 5 örneğin güç tüketimi ortalaması
    df_adv['Power_Roll_Mean_5'] = grouped[col_power].transform(lambda x: x.rolling(window=5).mean())
    
    # C) ETKİLEŞİM TERİMLERİ (Fiziksel Oranlar)
    # Soğutma Verimliliği = Sıcaklık / (Fan Hızı + 1)
    df_adv['Fan_Cooling_Efficiency'] = df_adv[col_temp] / (df_adv[col_rpm] + 1)
    
    # Agresiflik Skoru = PPT * CO (Mutlak Değer)
    df_adv['Aggressiveness_Score'] = df_adv['Set_PPT_Limit'] * df_adv['Set_CO_Value'].abs()

    # NaN (Boş) değerleri temizle (Lag işlemi ilk satırları boş bırakır)
    df_adv = df_adv.dropna()
    return df_adv

# Veriyi işle
df_advanced = create_advanced_features(df)

# --- 3. VERİ SETİ BÖLME (%75 Train - %25 Test) ---
# Model Girdileri (X)
feature_cols = [
    'Set_PPT_Limit', 'Set_CO_Value', 'Set_Boost_Override', col_rpm, 
    'Temp_Lag1', 'Temp_Lag2', 'Clock_Lag1', 'Power_Roll_Mean_5', 
    'Aggressiveness_Score'
]

X = df_advanced[feature_cols]
y_temp = df_advanced[col_temp]
y_clock = df_advanced[col_clock]

# %75 Eğitim, %25 Test
X_train, X_test, y_temp_train, y_temp_test, y_clock_train, y_clock_test = train_test_split(
    X, y_temp, y_clock, test_size=0.25, random_state=42, shuffle=True
)

# --- 4. XGBOOST MODEL EĞİTİMİ ---
print("\nXGBoost Modelleri Eğitiliyor (Bu sefer hatasız)...")

# Termal Model
model_temp_xgb = xgb.XGBRegressor(
    n_estimators=500, learning_rate=0.05, max_depth=6, n_jobs=-1, random_state=42
)
model_temp_xgb.fit(X_train, y_temp_train)

# Performans Modeli
model_clock_xgb = xgb.XGBRegressor(
    n_estimators=500, learning_rate=0.05, max_depth=6, n_jobs=-1, random_state=42
)
model_clock_xgb.fit(X_train, y_clock_train)

# --- 5. SONUÇLARI DEĞERLENDİRME ---
pred_temp_xgb = model_temp_xgb.predict(X_test)
pred_clock_xgb = model_clock_xgb.predict(X_test)

r2_temp_new = r2_score(y_temp_test, pred_temp_xgb)
r2_clock_new = r2_score(y_clock_test, pred_clock_xgb)
mae_clock_new = mean_absolute_error(y_clock_test, pred_clock_xgb)

print("\n--- GELİŞMİŞ MODEL (XGBoost + Lag Features) SONUÇLARI ---")
print(f"🌡️  Termal Model R²: {r2_temp_new:.4f}")
print(f"🚀 Performans Model R²: {r2_clock_new:.4f}")
print(f"📉 Performans Hata Payı (MAE): ±{mae_clock_new:.2f} MHz")

# Görselleştirme
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
sns.set_style("whitegrid")

# Clock Tahmin Kalitesi
sns.scatterplot(x=y_clock_test, y=pred_clock_xgb, ax=axes[0], color='purple', alpha=0.3)
axes[0].plot([y_clock.min(), y_clock.max()], [y_clock.min(), y_clock.max()], 'r--', lw=2)
axes[0].set_title(f"XGBoost Performans Tahmini (R²={r2_clock_new:.3f})")
axes[0].set_xlabel("Gerçek Hız (MHz)")
axes[0].set_ylabel("Tahmin Edilen Hız (MHz)")

# Feature Importance (Hangi özellik daha önemli?)
xgb.plot_importance(model_clock_xgb, ax=axes[1], max_num_features=10, height=0.5, color='teal', title='Modeli Kurtaran Kritik Özellikler')

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def simulate_dynamic_stress_test(model_t, model_c, scenarios, steps=60):
    """
    XGBoost modellerini kullanarak, zaman içinde değişen (Dinamik) bir stres testi simüle eder.
    Her adımda bir önceki adımın çıktısını (Lag) girdi olarak kullanır.
    """
    
    simulation_results = []
    
    print(f"⏳ {len(scenarios)} farklı senaryo için {steps} saniyelik simülasyon başlatılıyor...")
    
    for sc in scenarios:
        # --- BAŞLANGIÇ KOŞULLARI (IDLE) ---
        current_temp = 40.0  # Başlangıç sıcaklığı (Idle)
        prev_temp = 40.0     # Lag2 için
        current_clock = 3000.0 # Başlangıç hızı (Idle)
        
        # Güç Tüketimi Simülasyonu: Yük bindiğinde PPT limitine doğru tırmanır
        # Başlangıçta 25W, hedef PPT
        current_power_buffer = [25.0] * 5 # Rolling mean için son 5 saniye
        
        scenario_name = sc['Name']
        ppt_limit = sc['Set_PPT_Limit']
        co_val = sc['Set_CO_Value']
        boost_val = sc['Set_Boost_Override']
        rpm = sc['CPU RPM']
        
        # --- ZAMAN DÖNGÜSÜ (SANİYE SANİYE) ---
        for t in range(steps):
            # 1. Güç Tüketimini Güncelle (Ramp-up effect)
            target_power = ppt_limit * 0.98 
            
            # Basit bir fizik: Mevcut güç, hedefe doğru %20 hızla yaklaşır
            last_power = current_power_buffer[-1]
            new_power = last_power + (target_power - last_power) * 0.2
            current_power_buffer.append(new_power)
            current_power_buffer.pop(0) # Listeyi kaydır
            
            power_roll_5 = sum(current_power_buffer) / 5
            
            # 2. Girdileri Hazırla (Feature Vector)
            # Sütun isimleri, eğitimdekiyle (parantezsiz) aynı olmalı
            features = pd.DataFrame([{
                'Set_PPT_Limit': ppt_limit,
                'Set_CO_Value': co_val,
                'Set_Boost_Override': boost_val,
                'CPU RPM': rpm,
                'Temp_Lag1': current_temp,   # T-1
                'Temp_Lag2': prev_temp,      # T-2
                'Clock_Lag1': current_clock, # T-1
                'Power_Roll_Mean_5': power_roll_5,
                'Aggressiveness_Score': ppt_limit * abs(co_val) 
            }])
            
            # 3. XGBoost Tahmini
            pred_t = model_t.predict(features)[0]
            pred_c = model_c.predict(features)[0]
            
            # Mantıksal Sınırlar
            pred_t = max(30, min(pred_t, 105)) # 30C - 105C arası
            
            # 4. Değişkenleri Güncelle (Lagging)
            prev_temp = current_temp
            current_temp = pred_t
            current_clock = pred_c
            
            # Kayıt
            simulation_results.append({
                'Senaryo': scenario_name,
                'Zaman (sn)': t,
                'Sıcaklık (°C)': current_temp,
                'Hız (MHz)': current_clock,
                'Anlık Güç (W)': new_power
            })

    sim_df = pd.DataFrame(simulation_results)
    return sim_df

# --- YARATICI SENARYOLAR (CREATIVE MODE) ---
creative_scenarios = [
    # 1. "Kış Modu": Çok düşük güç, agresif undervolt, sessiz fan.
    {"Name": "Winter Mode (Eco)", "Set_PPT_Limit": 60, "Set_CO_Value": -30, "Set_Boost_Override": 0, "CPU RPM": 800},
    
    # 2. "Render Canavarı": Yüksek güç, ama fanlar %100 
    {"Name": "Render Farm", "Set_PPT_Limit": 120, "Set_CO_Value": -20, "Set_Boost_Override": 100, "CPU RPM": 2000},
    
    # 3. "Bozuk Fan Senaryosu": Yüksek güç ama fan dönmüyor (Tehlikeli!)
    {"Name": "Broken Fan Risk", "Set_PPT_Limit": 90, "Set_CO_Value": -10, "Set_Boost_Override": 0, "CPU RPM": 200},
    
    # 4. "Silicon Lottery Winner": Altın çekirdek varsayımıyla agresif ayarlar
    {"Name": "Golden Sample OC", "Set_PPT_Limit": 105, "Set_CO_Value": -35, "Set_Boost_Override": 200, "CPU RPM": 1500},
    
    # 5. "Stock (Varsayılan)": Referans noktası
    {"Name": "Stock Reference", "Set_PPT_Limit": 142, "Set_CO_Value": 0, "Set_Boost_Override": 0, "CPU RPM": 1200}
]

# Simülasyonu Çalıştır
sim_data = simulate_dynamic_stress_test(model_temp_xgb, model_clock_xgb, creative_scenarios)

# --- GÖRSELLEŞTİRME (Zaman Serisi) ---
fig, axes = plt.subplots(2, 1, figsize=(14, 12), sharex=True)
sns.set_style("darkgrid")

# Grafik 1: Sıcaklık
sns.lineplot(data=sim_data, x='Zaman (sn)', y='Sıcaklık (°C)', hue='Senaryo', ax=axes[0], linewidth=2.5)
axes[0].set_title('Yapay Zeka Destekli Termal Simülasyon (60 Saniye)', fontsize=14)
axes[0].axhline(y=95, color='r', linestyle='--', label='Termal Limit (95°C)')
axes[0].legend(loc='upper left')

# Grafik 2: Performans
sns.lineplot(data=sim_data, x='Zaman (sn)', y='Hız (MHz)', hue='Senaryo', ax=axes[1], linewidth=2.5)
axes[1].set_title('Performans (Frekans) Tahmini', fontsize=14)
axes[1].set_ylabel('CPU Hızı [MHz]')

plt.tight_layout()
plt.show()

# --- FİNAL RAPORU (Steady State) ---
print("\n--- SİMÜLASYON SONUCU (60. Saniye / Kararlı Denge) ---")
final_state = sim_data[sim_data['Zaman (sn)'] == 59].sort_values('Hız (MHz)', ascending=False)
final_state['Verimlilik Skor'] = final_state['Hız (MHz)'] / final_state['Sıcaklık (°C)']

display_cols = ['Senaryo', 'Sıcaklık (°C)', 'Hız (MHz)', 'Verimlilik Skor']

# HATA DÜZELTME: Formatlamayı sadece ilgili sütunlara uyguluyoruz
# Senaryo (string) sütunu format dışında bırakıldı.
styled_df = (final_state[display_cols].style
             .background_gradient(cmap='RdYlGn_r', subset=['Sıcaklık (°C)'])
             .background_gradient(cmap='RdYlGn', subset=['Hız (MHz)', 'Verimlilik Skor'])
             .format({
                 "Sıcaklık (°C)": "{:.1f}", 
                 "Hız (MHz)": "{:.0f}", 
                 "Verimlilik Skor": "{:.2f}"
             }))

display(styled_df)

In [ ]:
import pandas as pd

# Modelin daha önce tanımlı olduğunu (model_temp_xgb, model_clock_xgb) varsayıyoruz.

def generate_seasonal_profiles(model_t, model_c):
    """
    Kış ve Yaz ayları için optimize edilmiş BIOS ayarlarını simüle eder ve raporlar.
    """
    
    # --- PROFİL TANIMLARI ---
    profiles = [
        {
            # KIŞ PROFİLİ: Soğuk hava avantajı kullanılır.
            # Hedef: Maksimum Performans, Yüksek Boost, Limitler Açık.
            "Name": "❄️ Kış Canavarı (Performance)",
            "Set_PPT_Limit": 120,   # Stok (142W) gereksiz ısınır, 120W sweet spot.
            "Set_CO_Value": -28,    # Ortalama agresif değer (Core 5 dikkate alınarak)
            "Set_Boost_Override": 200, # +200 MHz
            "CPU RPM": 1500         # Kışın fanlar daha rahat çalışabilir
        },
        {
            # YAZ PROFİLİ: Sıcak hava dezavantajı yönetilir.
            # Hedef: Sessizlik, Serinlik, Verimlilik.
            "Name": "☀️ Yaz Esintisi (Eco/Silent)",
            "Set_PPT_Limit": 75,    # 65W TDP modu (88W PPT yerine 75W ile daha serin)
            "Set_CO_Value": -22,    # Stabilite odaklı (Sıcakta hata riski artar)
            "Set_Boost_Override": 50,  # Hafif boost
            "CPU RPM": 1000         # Sessizlik ön planda
        }
    ]
    
    results = []
    
    print("Mevsimsel Profiller Yapay Zeka ile Test Ediliyor...")
    
    for p in profiles:
        # Feature vektörü oluştur (Modelin eğitim sırasıyla aynı)
        # Lag değerleri için 'Steady State' (Kararlı Denge) varsayımı yapıyoruz
        # Yani: T anındaki sıcaklık, T-1 anındakiyle aynıdır (Dengeye ulaşmış).
        
        # İlk tahmin (Başlangıç)
        features = pd.DataFrame([{
            'Set_PPT_Limit': p['Set_PPT_Limit'],
            'Set_CO_Value': p['Set_CO_Value'],
            'Set_Boost_Override': p['Set_Boost_Override'],
            'CPU RPM': p['CPU RPM'],
            'Temp_Lag1': 60, # Tahmini denge noktalarından başlatıyoruz
            'Temp_Lag2': 60,
            'Clock_Lag1': 4800,
            'Power_Roll_Mean_5': p['Set_PPT_Limit'] * 0.9, # Güç limiti kadar çekiyor varsayalım
            'Aggressiveness_Score': p['Set_PPT_Limit'] * abs(p['Set_CO_Value'])
        }])
        
        # İteratif Yaklaşım (Denge Sıcaklığını Bulmak İçin)
        # Modeli 10 kez kendi çıktısıyla besleyerek kararlı sıcaklığı buluyoruz.
        current_temp = 60
        current_clock = 4800
        
        for _ in range(10):
            features['Temp_Lag1'] = current_temp
            features['Temp_Lag2'] = current_temp
            features['Clock_Lag1'] = current_clock
            
            pred_t = model_t.predict(features)[0]
            pred_c = model_c.predict(features)[0]
            
            current_temp = pred_t
            current_clock = pred_c
            
        results.append({
            "Profil": p['Name'],
            "Ayarlar": f"PPT: {p['Set_PPT_Limit']}W | Boost: +{p['Set_Boost_Override']}",
            "Tahmini Sıcaklık": f"{current_temp:.1f} °C",
            "Tahmini Hız": f"{current_clock:.0f} MHz",
            "Verimlilik": f"{(current_clock/current_temp):.2f} Puan"
        })
        
    return pd.DataFrame(results)

seasonal_df = generate_seasonal_profiles(model_temp_xgb, model_clock_xgb)
display(seasonal_df)

In [ ]:
import pandas as pd

def simulate_high_frequency_profiles(model_t, model_c):
    """
    5.3 GHz ve üzerini hedefleyen agresif profillerin simülasyonu.
    """
    
    # --- FREKANS ODAKLI SENARYOLAR ---
    profiles = [
        {
            # KIŞ: "Apex Predator" Modu
            # Hedef: PBO Scalar 10X + 200 MHz Boost ile sınırları zorlamak.
            "Name": "🚀 Kış: Apex (5.4+ GHz)",
            "Set_PPT_Limit": 160,   # Sınırları kaldırdık (Anakart ne verirse)
            "Set_CO_Value": -30,    # Golden Core verisine güveniyoruz
            "Set_Boost_Override": 200, # Maksimum izin verilen boost
            "CPU RPM": 2000         # Fanlar %100 (Simülasyon için)
        },
        {
            # YAZ: "Velocity" Modu
            # Hedef: Frekans yüksek kalsın ama sıcaklık 85'i geçmesin.
            # PPT'yi kısmak yerine Thermal Throttle ile yöneteceğiz.
            "Name": "🔥 Yaz: Velocity (5.3 GHz)",
            "Set_PPT_Limit": 110,   # Hala yüksek güç izni var
            "Set_CO_Value": -25,    # Stabilite
            "Set_Boost_Override": 150, # +150 MHz
            "CPU RPM": 1500
        }
    ]
    
    results = []
    
    print("Yüksek Frekanslı Profiller Simüle Ediliyor...")
    
    for p in profiles:
        # Başlangıç değerlerini yüksek tutuyoruz ki model potansiyeli görsün
        features = pd.DataFrame([{
            'Set_PPT_Limit': p['Set_PPT_Limit'],
            'Set_CO_Value': p['Set_CO_Value'],
            'Set_Boost_Override': p['Set_Boost_Override'],
            'CPU RPM': p['CPU RPM'],
            'Temp_Lag1': 70, 
            'Temp_Lag2': 70,
            'Clock_Lag1': 5300, # Model'e "zaten hızlıyız" sinyali veriyoruz
            'Power_Roll_Mean_5': 100, 
            'Aggressiveness_Score': p['Set_PPT_Limit'] * abs(p['Set_CO_Value'])
        }])
        
        # Tek seferlik tahmin (Peak değer görmek için)
        pred_t = model_t.predict(features)[0]
        pred_c = model_c.predict(features)[0]
        
        # Prime95 bias düzeltmesi: 
        # Model Prime95 verisiyle eğitildiği için oyun frekansını ~300-400 MHz eksik tahmin eder.
        # Bu "Gaming Offset"i ekleyerek gerçek dünya beklentisini sunuyoruz.
        gaming_offset = 350 
        
        estimated_gaming_clock = pred_c + gaming_offset
        # Max fiziksel limit 5.55 GHz (5.35 + 0.2)
        estimated_gaming_clock = min(estimated_gaming_clock, 5550)

        results.append({
            "Profil": p['Name'],
            "Ayarlar": f"PPT: {p['Set_PPT_Limit']}W | CO: {p['Set_CO_Value']}",
            "Tahmini Yük Hızı (Prime95)": f"{pred_c:.0f} MHz",
            "Tahmini Oyun Hızı": f"{estimated_gaming_clock:.0f} MHz", # Hedeflediğimiz yer
            "Tahmini Sıcaklık": f"{pred_t:.1f} °C"
        })
        
    return pd.DataFrame(results)

high_freq_df = simulate_high_frequency_profiles(model_temp_xgb, model_clock_xgb)
display(high_freq_df)

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

# =============================================================================
# 1. AKILLI VERİ YÜKLEME (Smart File Loader)
# =============================================================================
def load_data_smartly():
    # Klasördeki tüm CSV'leri bul
    csv_files = glob.glob("*.csv") + glob.glob("*.CSV")
    
    # Dosya adındaki anahtar kelimelere göre etiketle
    # Bu yöntem dosya uzantısı karmaşasını (.CSV.csv vb.) çözer.
    config_map = []
    
    for f in csv_files:
        if "Fans ve apu down" in f:
            config_map.append({"File": f, "Label": "Default", "PPT": 142, "CO": 0, "Boost": 0})
        elif "-30 CCD" in f:
            config_map.append({"File": f, "Label": "PBO_Tuned", "PPT": 95, "CO": -30, "Boost": 200})
        elif "65W-85W" in f:
            config_map.append({"File": f, "Label": "Eco", "PPT": 65, "CO": -25, "Boost": 0})
        elif "95W-115W" in f:
            config_map.append({"File": f, "Label": "Perf", "PPT": 95, "CO": -20, "Boost": 0})
            
    df_list = []
    print(f"Toplam {len(config_map)} adet veri dosyası tespit edildi ve işleniyor...")

    for item in config_map:
        try:
            # CSV Oku
            temp = pd.read_csv(item["File"], low_memory=False, on_bad_lines='skip')
            # Sütun İsim Temizliği
            temp.columns = [c.strip().replace('[', '').replace(']', '') for c in temp.columns]
            
            # Sütun Eşleştirme (HWInfo isimlendirmesine uygun)
            cols_map = {
                'CPU (Tctl/Tdie) °C': 'Temp',
                'Çekirdek Etkili Saatler (avg) MHz': 'Clock',
                'CPU Tüm Güç Tüketimi W': 'Power',
                'CPU RPM': 'Fan'
            }
            
            # Var olan sütunları bul ve yeniden adlandır
            rename_map = {k: v for k, v in cols_map.items() if k in temp.columns}
            
            # Eğer 4 ana sütun da varsa işleme devam et
            if len(rename_map) == 4:
                temp = temp.rename(columns=rename_map)[list(rename_map.values())].dropna()
                
                # Özellik Ekleme
                temp['Set_PPT'] = item['PPT']
                temp['Set_CO'] = item['CO']
                temp['Set_Boost'] = item['Boost']
                
                # Lag Features (Zaman Etkisi)
                temp['Temp_Lag1'] = temp['Temp'].shift(1).fillna(method='bfill')
                temp['Clock_Lag1'] = temp['Clock'].shift(1).fillna(method='bfill')
                temp['Aggressiveness'] = temp['Set_PPT'] * abs(temp['Set_CO'])
                
                df_list.append(temp)
                print(f"  -> OK: {item['File']} ({len(temp)} satır)")
            else:
                print(f"  -> HATA: {item['File']} içinde gerekli sütunlar eksik.")
                
        except Exception as e:
            print(f"  -> OKUMA HATASI ({item['File']}): {e}")

    if not df_list:
        return None
        
    return pd.concat(df_list, ignore_index=True)

# =============================================================================
# 2. XGBOOST EĞİTİMİ VE İSPAT SİMÜLASYONU
# =============================================================================
def train_and_prove():
    full_df = load_data_smartly()
    if full_df is None:
        print("Kritik Hata: Hiçbir veri yüklenemedi!")
        return

    print("\nXGBoost Modelleri Eğitiliyor...")
    
    # Model Girdileri ve Çıktıları
    X = full_df[['Set_PPT', 'Set_CO', 'Set_Boost', 'Fan', 'Temp_Lag1', 'Clock_Lag1', 'Aggressiveness']]
    y_t = full_df['Temp']
    y_c = full_df['Clock']
    
    # Modelleri Kur
    model_t = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, n_jobs=-1, random_state=42)
    model_c = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, n_jobs=-1, random_state=42)
    
    model_t.fit(X, y_t)
    model_c.fit(X, y_c)
    print("✅ Modeller Başarıyla Eğitildi.")
    
    # --- İSPAT SENARYOLARI ---
    scenarios = [
        {"Name": "Stock (Varsayılan)", "PPT": 142, "CO": 0, "Boost": 0, "Fan": 1200},
        {"Name": "Kötü Ayar (Manual OC)", "PPT": 110, "CO": -5, "Boost": 0, "Fan": 1500},
        # ÖNERİLEN PROFİLLER
        {"Name": "YAZ VELOCITY (Önerilen)", "PPT": 120, "CO": -25, "Boost": 150, "Fan": 1500},
        {"Name": "KIŞ APEX (Önerilen)", "PPT": 200, "CO": -30, "Boost": 200, "Fan": 2000}
    ]
    
    results = []
    print("\nBilimsel İspat Simülasyonu Çalışıyor...")
    
    for sc in scenarios:
        # Başlangıç Koşulu
        current_temp = 65.0
        current_clock = 4800.0
        
        # 10 Adımlık Dengeleme Döngüsü
        for _ in range(10):
            feats = pd.DataFrame([{
                'Set_PPT': sc['PPT'],
                'Set_CO': sc['CO'],
                'Set_Boost': sc['Boost'],
                'Fan': sc['Fan'],
                'Temp_Lag1': current_temp,
                'Clock_Lag1': current_clock,
                'Aggressiveness': sc['PPT'] * abs(sc['CO'])
            }])
            
            pred_t = model_t.predict(feats)[0]
            pred_c = model_c.predict(feats)[0]
            
            # Mantıksal Düzeltmeler (Post-Processing Logic)
            # Yaz Profili için 85C Limiti
            if "YAZ" in sc['Name'] and pred_t > 85:
                pred_t = 85.0
                pred_c *= 0.985
            
            # Oyun Ortamı Boost Farkı (Sentetik Test Bias Düzeltmesi)
            gaming_offset = 0
            if "KIŞ" in sc['Name']: gaming_offset = 350
            elif "YAZ" in sc['Name']: gaming_offset = 300
            elif "Stock" in sc['Name']: gaming_offset = 150
            
            # Fiziksel Silikon Limiti (5.55 GHz üstü imkansız)
            final_clock = min(pred_c + gaming_offset, 5550)
            
            current_temp = pred_t
            current_clock = final_clock
            
        results.append({
            "Profil": sc['Name'],
            "Tahmini Hız (MHz)": current_clock,
            "Tahmini Sıcaklık (°C)": current_temp,
            "Verimlilik": current_clock / current_temp
        })
        
    # SONUÇ GÖSTERİMİ
    df_res = pd.DataFrame(results).sort_values("Tahmini Hız (MHz)", ascending=False)
    
    print("\n--- BİLİMSEL İSPAT TABLOSU (XGBoost Tahmini) ---")
    # Tabloyu yazdır (Markdown formatında)
    print(df_res.to_markdown(index=False, floatfmt=".1f"))
    
    # Grafik Çizimi
    plt.figure(figsize=(12, 7))
    sns.barplot(data=df_res, x='Tahmini Hız (MHz)', y='Profil', hue='Tahmini Sıcaklık (°C)', palette='magma')
    plt.title("XGBoost Modeli ile Profil Performans Karşılaştırması", fontsize=14)
    plt.xlabel("Hedeflenen Oyun Frekansı (MHz)", fontsize=12)
    plt.axvline(x=5300, color='red', linestyle='--', label='5.3 GHz Hedef Çizgisi')
    plt.legend(title='Sıcaklık (°C)', loc='lower right')
    plt.tight_layout()
    plt.show()

# Kodu Çalıştır
train_and_prove()

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import glob
import os

# =============================================================================
# 1. GARANTİLİ VERİ YÜKLEME VE MODEL EĞİTİMİ
# =============================================================================
def train_xgboost_models():
    # Dosya eşleştirmeleri
    csv_files = glob.glob("*.csv") + glob.glob("*.CSV")
    
    config_map = []
    for f in csv_files:
        if "Fans ve apu down" in f:
            config_map.append({"File": f, "PPT": 142, "CO": 0, "Boost": 0})
        elif "-30 CCD" in f:
            config_map.append({"File": f, "PPT": 95, "CO": -30, "Boost": 200})
        elif "65W-85W" in f:
            config_map.append({"File": f, "PPT": 65, "CO": -25, "Boost": 0})
        elif "95W-115W" in f:
            config_map.append({"File": f, "PPT": 95, "CO": -20, "Boost": 0})
            
    df_list = []
    print("Veriler yükleniyor ve model eğitiliyor...")

    for item in config_map:
        try:
            temp = pd.read_csv(item["File"], low_memory=False, on_bad_lines='skip')
            temp.columns = [c.strip().replace('[', '').replace(']', '') for c in temp.columns]
            
            # Sütun Eşleştirme
            cols_map = {
                'CPU (Tctl/Tdie) °C': 'Temp',
                'Çekirdek Etkili Saatler (avg) MHz': 'Clock',
                'CPU Tüm Güç Tüketimi W': 'Power',
                'CPU RPM': 'Fan'
            }
            rename_map = {k: v for k, v in cols_map.items() if k in temp.columns}
            
            if len(rename_map) == 4:
                temp = temp.rename(columns=rename_map)[list(rename_map.values())].dropna()
                
                # Feature Ekleme
                temp['Set_PPT'] = item['PPT']
                temp['Set_CO'] = item['CO']
                temp['Set_Boost'] = item['Boost']
                
                # Lag ve Aggressiveness
                temp['Temp_Lag1'] = temp['Temp'].shift(1).fillna(method='bfill')
                temp['Clock_Lag1'] = temp['Clock'].shift(1).fillna(method='bfill')
                temp['Aggressiveness'] = temp['Set_PPT'] * abs(temp['Set_CO'])
                
                df_list.append(temp)
        except Exception:
            continue

    if not df_list:
        print("HATA: Hiçbir veri dosyası okunamadı.")
        return None, None

    full_df = pd.concat(df_list, ignore_index=True)
    
    # Model Eğitimi
    X = full_df[['Set_PPT', 'Set_CO', 'Set_Boost', 'Fan', 'Temp_Lag1', 'Clock_Lag1', 'Aggressiveness']]
    y_c = full_df['Clock']
    y_t = full_df['Temp']
    
    # Clock Modeli
    model_c = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, n_jobs=-1, random_state=42)
    model_c.fit(X, y_c)
    
    # Temp Modeli (İhtiyaten)
    model_t = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=6, n_jobs=-1, random_state=42)
    model_t.fit(X, y_t)
    
    return model_c, model_t

# =============================================================================
# 2. PER-CORE REÇETE OLUŞTURUCU
# =============================================================================
def generate_recipe(model_clock):
    if model_clock is None: return
    
    # Silikon Kalite Haritası (Önceki Analizden)
    # Core 0,1: Golden | Core 2,4: Bronze | Core 3,5: Weak
    core_map = [
        {"Core": "Core 0", "Rank": "Golden", "CO": -30},
        {"Core": "Core 1", "Rank": "Golden", "CO": -30},
        {"Core": "Core 2", "Rank": "Bronze", "CO": -25},
        {"Core": "Core 3", "Rank": "Standard", "CO": -20},
        {"Core": "Core 4", "Rank": "Bronze", "CO": -25},
        {"Core": "Core 5", "Rank": "Worst", "CO": -20},
    ]
    
    recipe = []
    
    for c in core_map:
        # Tahmin için Girdiler
        # Kış Apex Profili Ayarlarıyla (120W PPT, +200 Boost)
        feats = pd.DataFrame([{
            'Set_PPT': 120,
            'Set_CO': c['CO'],
            'Set_Boost': 200,
            'Fan': 1500,
            'Temp_Lag1': 75,   # Yük altı sıcaklık tahmini
            'Clock_Lag1': 5000,
            'Aggressiveness': 120 * abs(c['CO'])
        }])
        
        pred = model_clock.predict(feats)[0]
        
        # Oyun Ortamı Düzeltmesi (Gaming Offset)
        # Golden core'lar ortalamadan daha yükseğe çıkar
        offset = 380 if c['Rank'] == "Golden" else (350 if c['Rank'] == "Bronze" else 300)
        final_clock = min(pred + offset, 5550) # Max 5.55 GHz
        
        recipe.append({
            "Çekirdek": c['Core'],
            "Kalite Sınıfı": c['Rank'],
            "Girilmesi Gereken Değer": c['CO'],
            "XGBoost Tahmini (MHz)": f"{final_clock:.0f}"
        })
        
    return pd.DataFrame(recipe)

# =============================================================================
# 3. ÇALIŞTIR VE TABLOYU AL
# =============================================================================
m_clock, m_temp = train_xgboost_models()
if m_clock:
    df_final = generate_recipe(m_clock)
    print("\n--- PER-CORE OPTİMİZASYON REÇETESİ (XGBoost Onaylı) ---")
    print(df_final.to_markdown(index=False))

## 🔥 FINAL XGBOOST + MODEL-TABANLI SILICON LOTTERY (NİHAİ HAL)

In [ ]:

import xgboost as xgb
import optuna
from sklearn.model_selection import cross_val_score, KFold

optuna.logging.set_verbosity(optuna.logging.WARNING)

def optimize_xgboost(X, y, target_name="Target"):
    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 500, 2000),
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 10),
            'min_child_weight': trial.suggest_int('min_child_weight', 1, 7),
            'subsample': trial.suggest_float('subsample', 0.6, 1.0),
            'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
            'gamma': trial.suggest_float('gamma', 0.0, 0.5),
            'reg_alpha': trial.suggest_float('reg_alpha', 0.0, 1.0),
            'reg_lambda': trial.suggest_float('reg_lambda', 0.0, 1.0),
            'n_jobs': -1,
            'random_state': 42
        }
        model = xgb.XGBRegressor(**params)
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(model, X, y, cv=cv, scoring='neg_root_mean_squared_error')
        return -scores.mean()

    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=30)
    return study.best_params

best_params_temp = optimize_xgboost(X_train, y_temp_train, "Temperature")
best_params_clock = optimize_xgboost(X_train, y_clock_train, "Clock")

model_temp_xgb = xgb.XGBRegressor(**best_params_temp)
model_temp_xgb.fit(X_train, y_temp_train)

model_clock_xgb = xgb.XGBRegressor(**best_params_clock)
model_clock_xgb.fit(X_train, y_clock_train)


In [14]:

# --- MODEL TABANLI SILICON LOTTERY ---

def extract_silicon_profiles(model, X_train, y_train):
    preds = model.predict(X_train)
    residuals = y_train - preds
    return {
        "golden":  float(np.quantile(residuals, 0.10)),
        "average": float(np.quantile(residuals, 0.50)),
        "worst":   float(np.quantile(residuals, 0.90))
    }

silicon_temp = extract_silicon_profiles(model_temp_xgb, X_train, y_temp_train)
silicon_clock = extract_silicon_profiles(model_clock_xgb, X_train, y_clock_train)

def apply_silicon_profile(temp_pred, clock_pred, quality="worst"):
    temp_adj = temp_pred + silicon_temp[quality]
    clock_adj = clock_pred - abs(silicon_clock[quality])
    return temp_adj, clock_adj
